# 3.2: Inspecting the `data` directory

`run_prepare_data.sh` will generate a new directory,`data`, that contains many of the files required for the `ASR` pipeline.  We will inspect its contents below.

**Note:** The official `kaldi` documentation has a more detailed explanation of these files [here](http://kaldi-asr.org/doc/data_prep.html).  Just beware that **some** files explained there are not relevant to our pipeline.

In [ ]:
ls data

## `data/{train|test}_dir`

These directories contain four files for each of the subsets, `train` and `test` (assuming you set up a configuration with both when you ran `run_prepare_data.sh`, which we did).

In [5]:
ls data/train_dir

spk2utt  text  utt2spk  wav.scp


### `wav.scp`

This file maps each `audio basename` to its full path.  It takes the format:

```
[audio-basename] [full/path/to/audio]
[audio-basename] [full/path/to/audio]
[audio-basename] [full/path/to/audio]
```

In [ ]:
head -n5 data/train_dir/wav.scp

**Note:** `kaldi` allows the second argument of this file to be a `piped` command.  See this example from the official documentation:

```
s5# head -3 data/train/wav.scp
sw02001-A /home/dpovey/kaldi-trunk/tools/sph2pipe_v2.5/sph2pipe -f wav -p -c 1 /export/corpora3/LDC/LDC97S62/swb1/sw02001.sph |
sw02001-B /home/dpovey/kaldi-trunk/tools/sph2pipe_v2.5/sph2pipe -f wav -p -c 2 /export/corpora3/LDC/LDC97S62/swb1/sw02001.sph |
```

In this case `/home/dpovey/kaldi-trunk/tools/sph2pipe_v2.5/sph2pipe -f wav -p -c 1 /export/corpora3/LDC/LDC97S62/swb1/sw02001.sph` is a command that will convert a `.sph` file to `wav`.  The final character, `|`, acts as if it were `piping` the output to a file location.  

In our case, we explicitly converted the audio as a preprocessing step, but one can easily follow this approach if it's important that the original audio file not be modified.

### `text`

This file is a subset of the `transcripts` file supplied to `run_prepare_data.sh` that contains **only** the transcripts of the utterances in that particular subset.

In [ ]:
head -n5 data/train_dir/text

### `utt2spk`

This file maps each utterance to a speaker.  Having speaker information becomes important later when we try to make certain speaker-level adaptations to the model (in an attempt to generalize the characteristics of a particular speaker). 

The file takes the following format:

```
[utterance-id] [speaker-id]
[utterance-id] [speaker-id]
[utterance-id] [speaker-id]
```

In [4]:
head -n5 data/train_dir/utt2spk

1272-128104-0000 1272-128104-0000
1272-128104-0001 1272-128104-0001
1272-128104-0002 1272-128104-0002
1272-128104-0003 1272-128104-0003
1272-128104-0004 1272-128104-0004


**Note:** If we weren't to have any information about speakers in our dataset, we could simply make the `speaker-id` the same as the `utterance-id`, which would result in `n` unique speakers (where `n` is the number of utterances in the subset).

And so you can see from the file, this `kaldi` pipeline works under the assumption that we do **not** have any information about speakers.  This is not particularly true in the case of the `librispeech` dataset, however.

One of the files created by an intermediate step during `1:1 Downloading...` created an `utt2spk` file for each of the original `librispeech` subsets.  They are stored in `raw_data/Librispeech/[subset]_data`.

In [5]:
head -n5 raw_data/LibriSpeech/dev-clean_data/utt2spk

1272-128104-0000 1272-128104
1272-128104-0001 1272-128104
1272-128104-0002 1272-128104
1272-128104-0003 1272-128104
1272-128104-0004 1272-128104


Here you can see that the `utterance-id` convention is such that the last portion `000*` refers to a segment of a larger audio directory, all spoken by the same person.

**Note:** We will stick with our current `utt2spk` for the sake of continuity in our pipeline, but at a later date, we can revisit this and see how performance changes (improves?) when using an `utt2spk` file that takes speaker information into account. 

### `spk2utt`

This file is simply a reverse mapping of `utt2spk`.  Instead of mapping an uttreance to a speaker, this file maps a speaker to all of her or his spoken utterances.  It thus takes the format:

```
[speaker-id] [utterance-id_1] [utterance-id_2] [utterance-id_3] ... [utterance-id_n]
[speaker-id] [utterance-id_1] [utterance-id_2] [utterance-id_3] ... [utterance-id_n]
[speaker-id] [utterance-id_1] [utterance-id_2] [utterance-id_3] ... [utterance-id_n]
```

In our case, this will be identicial to `utt2spk` since we are not taking speaker information into account (and we thus have a unique speaker for each utterance).

In [7]:
head -n5 data/train_dir/spk2utt

1272-128104-0000 1272-128104-0000
1272-128104-0001 1272-128104-0001
1272-128104-0002 1272-128104-0002
1272-128104-0003 1272-128104-0003
1272-128104-0004 1272-128104-0004


But looking at the `spk2utt` in `raw_data/LibriSpeech/[subset]`, we can see what a `spk2utt` would like that **does** take speaker information into account.

In [9]:
head -n2 raw_data/LibriSpeech/dev-clean_data/spk2utt

1272-128104 1272-128104-0000 1272-128104-0001 1272-128104-0002 1272-128104-0003 1272-128104-0004 1272-128104-0005 1272-128104-0006 1272-128104-0007 1272-128104-0008 1272-128104-0009 1272-128104-0010 1272-128104-0011 1272-128104-0012 1272-128104-0013 1272-128104-0014
1272-135031 1272-135031-0000 1272-135031-0001 1272-135031-0002 1272-135031-0003 1272-135031-0004 1272-135031-0005 1272-135031-0006 1272-135031-0007 1272-135031-0008 1272-135031-0009 1272-135031-0010 1272-135031-0011 1272-135031-0012 1272-135031-0013 1272-135031-0014 1272-135031-0015 1272-135031-0016 1272-135031-0017 1272-135031-0018 1272-135031-0019 1272-135031-0020 1272-135031-0021 1272-135031-0022 1272-135031-0023 1272-135031-0024


### `segments` (optional)

This file does **not** appear in our pipeline when using `librispeech` data, but if were accessing audio that was **not** already segmented, we would have supplied a master `segments` file to `run_prepare_data.sh`, which would have generated a `{train|test}`-specific `segments` file here.  It would take the following format:

```
[audio-basename] [utterance-id] [utterance-start] [utterance-stop]
[audio-basename] [utterance-id] [utterance-start] [utterance-stop]
[audio-basename] [utterance-id] [utterance-start] [utterance-stop]
```

## `data/local`

This directory is an intermediate (essentially, `temp`) directory used for housing files as they are manipulated and/or built for later use.  All the **important** files will appear in another subdirectory of `data`, so we won't spend too much time on the items here.

### `waves.{train|test}`

These files are simply a list of the audio files that belong to the `train` and/or `test` subsets.

In [ ]:
head -n5 data/local/waves.train
echo ...
head -n5 data/local/waves.test

### `lm_tg.arpa`

This is a modified version of the `language model` that you supplied as an argument to `run_prepare_data.sh` with any `n-gram` containing `<UNK>` removed.  This will ensure that our model will **never** predict `<UNK>` when decoding.

In [ ]:
diff data/local/lm_tg.arpa raw_data/3-gram.pruned.3e-7.arpa | head -n10

### `data/local/dict`

This directory contains files pertaining to the `lexicon`.

In [ ]:
ls data/local/dict

#### `lexicon.txt`

This is just a local copy of the `lexicon` you supplied to `run_prepare_data.sh`

In [13]:
diff data/local/dict/lexicon.txt raw_data/librispeech-lexicon.txt

**Note:** `kaldi` will do this alot: it will **copy** files to a `local` location and then call those files from that **local** location (as opposed to their original locations.  This is inefficient from a disk space perspective in my opinion, but it would be a tremendous amount of work to parameterize all the scripts to take a location.  And disk space is cheap...So we will suffer it.

#### `lexiconp.txt`

This is a form of the `lexicon` with an additional value for each word: the probability of that pronunciation.

In [ ]:
cat data/local/dict/lexiconp.txt | grep INDIRECTLY

This allows for you to provide a `lexicon` that not only provides alternative pronunciations, but weights them according to their likelihood.  In our case, however, we don't have data to support the setting of those values, so all pronunciations are equally weighted at `1.0` (yes, it probably should be `.5` and `.5`, but `kaldi` is OK with `1.0` all the different pronunciations of a given word).

#### `{non}silence_phones.txt` and `optional_silence.txt`

`silence_phones.txt` and `nonsilence_phones.txt` simply separate the `phones` we supplied to `run_data_prepare.sh` into those that refer to `silence` and those that don't.  In our case, the only `silence` phone is `SIL`.

In [ ]:
cat data/local/dict/silence_phones.txt

`optional_silence.txt` contains the value for a `phone` we will use to identify the `silence` between words.  The official `kaldi` documentation linked above doesn't go into much detail as to why this is used.

In [ ]:
cat data/local/dict/optional_silence.txt

### `data/local/lang`

This directory is truly only a `temp` `directory` used in the building of `data/lang`.  So we will skip investigating this directory, and move on to `data/lang`. 

## `data/lang`

This directory will contain all the files needed to utilize the `language model` (how `kaldi` accesses the `ARPA`-format `language model` during decoding will be discussed later).

### `words.txt`

This file is a mapping of each word in our `lexicon` to an index.

**Note:** `kaldi` does this often.  You will see many examples of files like this that map something to an index for efficiency.

In [15]:
head -n5 data/lang/words.txt
echo ...
tail -n5 data/lang/words.txt

<eps> 0
<unk> 1
A 2
A''S 3
A'BODY 4
...
ZZZ 200000
ZZZZ 200001
#0 200002
<s> 200003
</s> 200004


You'll notice a few added "words" to this list (more on each of these in later weeks):

 - `<eps>`: a faux-word used to model the "space" between words in a later step
 - `#0`: a faux-word used to allow our `finite state transducer (FST)` to function properly
 - `<s>`: an faux-word used to model the "start of an utterance" in a later step
 - `</s>`: an faux-word used to model the "end of an utterance" in a later step

### `phones.txt`

This file is a mapping of each phone (from the list that we supplied to `run_prepare_data.sh`) to an index.

In this case, there are two additional differences between this file and the one we supplied to `run_prepare_data.sh`:

 - an additional `<eps>` phone is added (more about the role this plays later)
 - each phone is converted to `BIES` notation, which identifies where the phone occurs in the word 
   - `B`eginning
   - `I`nside
   - `E`nd
   - `S`olo (the word is made up of only this phone)

In [12]:
head data/lang/phones.txt

<eps> 0
SIL 1
SIL_B 2
SIL_E 3
SIL_I 4
SIL_S 5
AA0_B 6
AA0_E 7
AA0_I 8
AA0_S 9


### `{arpa}_oov.txt`

These two files identify any words in the `lexicon` that do **not** appear in our `language model` (`OOV` = out of vocabulary).

**Note:** This will hopefully become evident later, but this would cause a significant problem for our `ASR` model, and so we took steps during the preparation stage to identify any such words.

In [16]:
cat data/lang/arpa_oov.txt
echo ...
cat data/lang/oov.txt

<UNK>
...
<unk>


In our case, since our `language model` and `lexicon` were both built directly from the same source, it shouldn't be surprising to see an "empty" list.  But you can imagine a situation where your `language model` would be built from a very large corpus, and while there are ways to automatically generate (approximate) pronunciations of words (see [here](https://www-i6.informatik.rwth-aachen.de/web/Software/g2p.html) for one such program), it may not be an efficient step in the data preparation to worry about ensuring all words are present in both files.

In a later step, we will automatically **remove** any such `OOV` words from the final `FST`.

### `oov.int`

This file is a representation of `oov.txt` using indices instead of strings.

**Note:** `kaldi` will do this a lot.  And the filetypes (`.txt` v. `int`) will be an indication of which form to expect.  

In [17]:
cat data/lang/oov.int

1


### `topo`

### `L.fst` and `L_disambig.fst`

In [23]:
fstinfo --help

bash: fstinfo: command not found


: 127

In [22]:
/home/kaldi/tools/openfst/bin/fstinfo --help

Prints out information about an FST.

  Usage: /home/kaldi/tools/openfst/bin/fstinfo [in.fst]

PROGRAM FLAGS:

  --arc_filter: type = string, default = "any"
  Arc filter: one of: "any", "epsilon", "iepsilon", "oepsilon"; this only affects the counts of (co)accessible states, connected states, and (strongly) connected components
  --fst_verify: type = bool, default = true
  Verify FST sanity
  --info_type: type = string, default = "auto"
  Info format: one of: "auto", "long", "short"
  --pipe: type = bool, default = false
  Send info to stderr, input to stdout
  --test_properties: type = bool, default = true
  Compute property values (if unknown to FST)

LIBRARY FLAGS:

Flags from: flags.cc
  --help: type = bool, default = false
  show usage information
  --helpshort: type = bool, default = false
  show brief usage information
  --tmpdir: type = string, default = "/tmp"
  temporary directory
  --v: type = int32, default = 0
  verbosity level

Flags from: fst.cc
  --fst_align: type = bo

: 1

### `data/lang/phones`

This directory...